## Deelvraag 1
Data-preparatie: Hoe kan cellijn-data (mutaties, methylatie, drug-respons) worden verzameld, opgeschoond en geschikt gemaakt voor machine learning?

Situatie: Ik heb 3 verschillende bestanden die moeten worden samengevoegd:

- OmicsSomaticMutations.csv > bevat de mutatieprofielen
- Drug_sensitivity_AUC_(PRISM_Repurposing_Secondary_Screen)_subsetted.csv > bevat de response (AUC-waarden)
- CCLE_RRBS_TSS1kb_20181022.txt > bevat de methylatieprofielen
- 
Ik weet welke feature ik uit de bestanden nodig heb, maar nog niet de precieze kolomnamen. Het lijkt me daarom de beste aanpak om eerst de bestanden apart te verkennen en te filteren en om ze daarna pas samen te voegen. Als je meteen alles merge't, krijg je een gigantische rommelige dataset met heel veel kolommen die je niet nodig hebt. Dat kan er voor zorgen dat je het overzicht verliest. Door per bestand eerst te filteren p de relevante featuer, hou je kleinere, schone tabellen over die je daarna netjes kunt samenvoegen. Het scheelt ook rekenkracht en tijd.



#### Stap 1 - Data inladen en verkennen (met pandas)

Doel:
- De volgende datasets netjeds inladen als DataFrame
- Snappen wat er in zitL kolomnamen, grootte, missende waarden, dtypes
- Klaarmaken voor volgende stap (combineren)

In [12]:
# importeren libraries
from pathlib import Path
import pandas as pd

PROJECT_ROOT = Path.cwd().parent.parent
print("Project root:", PROJECT_ROOT)

# pad naar databestand
methylation_path = PROJECT_ROOT / "data" / "raw" / "CCLE_RRBS_TSS1kb_20181022.txt"

# check of bestand bestaat
if not methylation_path.exists():
    raise FileNotFoundError(f"Bestand niet gevonden: {methylation_path}")
print("Data bestand gevonden!")

# klein stukje data inladen om te controleren
methylation_df = pd.read_csv(methylation_path, sep='\t', nrows=5)
print("Data sample:")
print(methylation_df.head())

# grootte van de dataset achterhalen
print("\nAantal rijen en kolomme:", methylation_df.shape)

# kolomnamen
print("\nKolommen:", methylation_df.columns.tolist())

# basisstatistieken van numerieke kolommen
print("\nBasisstatistieken:")
print(methylation_df.describe())

    

Project root: /Users/jalisavanderzeeuw/ml_oncoresponse
Data bestand gevonden!
Data sample:
                    locus_id  \
0  SGIP1_1_66998638_66999638   
1  SGIP1_1_66998251_66999251   
2  AZIN2_1_33545713_33546713   
3  AZIN2_1_33546778_33547778   
4  AGBL4_1_50489626_50490626   

                                      CpG_sites_hg19  avg_coverage  \
0  1:66998970;1:66998973;1:66998993;1:66999404;1:...         25.00   
1                   1:66998970;1:66998973;1:66998993          8.27   
2  1:33546151;1:33546209;1:33546210;1:33546385;1:...        326.58   
3  1:33546783;1:33546788;1:33546795;1:33546797;1:...        480.54   
4  1:50489632;1:50489641;1:50489671;1:50489677;1:...        263.36   

   DMS53_LUNG SW1116_LARGE_INTESTINE P3HR1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE  \
0     0.00000                0.00000                                  0.75000   
1     0.00000                    NaN                                      NaN   
2     0.00729                0.03232                

In [15]:
# filteren op long-cellijnen
lung_columns = [col for col in methylation_df.columns if col.endswith("_LUNG")]

# de eerste drie metadata-kolommen wil ik behouden
base_cols = ['locus_id', 'CpG_sites_hg19', 'avg_coverage']

# maak een nieuw dataframe met alleen long cellijnen
lung_df = methylation_df[base_cols + lung_columns]

print(f"Aantal kolommen in long dataset: {len(lung_df.columns)}")
print("Voorbeeld kolomnamen:", lung_df.columns[:10].tolist())

# bekijk paar rijen voor controle
lung_df.head()

Aantal kolommen in long dataset: 156
Voorbeeld kolomnamen: ['locus_id', 'CpG_sites_hg19', 'avg_coverage', 'DMS53_LUNG', 'NCIH1184_LUNG', 'NCIH2227_LUNG', 'RERFLCAD2_LUNG', 'NCIH2347_LUNG', 'NCIH2087_LUNG', 'NCIH2085_LUNG']


,locus_id,CpG_sites_hg19,avg_coverage,DMS53_LUNG,NCIH1184_LUNG,NCIH2227_LUNG,RERFLCAD2_LUNG,NCIH2347_LUNG,NCIH2087_LUNG,NCIH2085_LUNG,...,NCIH1944_LUNG,NCIH2405_LUNG,BEN_LUNG,NCIH727_LUNG,HCC2935_LUNG,LCLC103H_LUNG,LCLC97TM1_LUNG,HOP62_LUNG,EKVX_LUNG,HOP92_LUNG
0,SGIP1_1_66998638_66999638,1:66998970;1:66998973;1:66998993;1:66999404;1:...,25.00,0.00000,0.11864,0.04545,NaN,0.25000,0.86441,1.00000,...,0.10344,NaN,0.90907,NaN,NaN,NaN,1.00000,NA,NA,0.66667
1,SGIP1_1_66998251_66999251,1:66998970;1:66998973;1:66998993,8.27,0.00000,0.22579,NaN,NaN,NaN,0.97871,NaN,...,0.11110,NaN,NaN,NaN,NaN,NaN,NaN,NA,NA,0.66667
2,AZIN2_1_33545713_33546713,1:33546151;1:33546209;1:33546210;1:33546385;1:...,326.58,0.00729,0.22553,0.04560,0.14577,0.11487,0.17708,0.15413,...,0.02693,0.00697,0.07692,0.05825,0.24294,0.01045,0.03746,NA,NA,0.00046
3,AZIN2_1_33546778_33547778,1:33546783;1:33546788;1:33546795;1:33546797;1:...,480.54,0.22276,0.10735,0.03326,0.09589,0.21004,0.35571,0.03926,...,0.04961,0.00380,0.01200,0.12764,0.36441,0.24371,0.23342,NA,NA,0.05159
4,AGBL4_1_50489626_50490626,1:50489632;1:50489641;1:50489671;1:50489677;1:...,263.36,0.00000,0.07346,0.01246,0.00531,0.07422,0.60335,0.00000,...,0.00130,0.11082,0.01485,0.10371,0.01515,0.05079,0.86448,NA,NA,0.54553


ik heb nu 156 kolommen waarvan 153 long cellijnen zijn. 
elke rij = één locus (DNA-positie)
elke kolom = één cellijn
de waarde = methylatiepercentage van dat locus in die cellijn

De cellijnen moeten naar rijen worden gedraaid

CpG_sites_hg19 = genomische voordinaten van CpG-sites in het Hg19-genoom
avg_coverage = maat voor de betrouwbaarheid van de methylatie-metingen op dat locus

TSS=Transcription Start Site, het beginpunt van een gen waar transcriptie start.
±1kb betekent dat de gemeten regio 1 kilobase (1000bp) upstream en downstream van de TSS ligt
dus direct rond het begin van het gen, inclusief een stukje voor (promotorgebied) en erna (eerste exon/regio lakbij TSS)

- Upstream van TSS (~1000 bp): dit is meestal de promoter. Methylatie hier kan de transcriptie van het gen onderdrukken.
- Downstream van TSS (~1000 bp): dit is vaak het 5’ einde van het gen, inclusief exon 1 of de 5’ UTR. Methylatie hier kan ook effect hebben op genexpressie. methylatie in deze regio kan het initiatiefase van transcriptie veranderen, waardoor het gen minder of anders wordt afgelezen

Nadenken over avg_coverage. minimum instellen. te lagen waarden weglaten omdat de methylatiepercentages dan ruis bevatten (in de literatuur 10>)
laat ook zien want de impact is van die drempel.


#### Stap 2 - Data combineren (met pandas)
nodig: merge, concat

#### Stap 3 - Opschonen
scikit-learn kan geen lege waarden of niet-numerieke kolommen gebruiken. alles moet getallen zijn en het moet één nette matrix zijn.